In [10]:
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, AveragePooling2D, Dropout, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import os


In [2]:
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

In [3]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [4]:
#normalize the data
train_x = train_x.astype('float32')/255
test_x = test_x.astype('float32')/255

In [5]:
test_y.shape

(10000, 1)

In [6]:
#encode the labels to vectors
train_y = keras.utils.to_categorical(train_y, 10)
test_y = keras.utils.to_categorical(test_y, 10)

In [22]:
#define the model
def MiniModel(input_shape):
  images = Input(input_shape)


  net = Conv2D(filters = 64, kernel_size = [3,3], strides = [1,1], padding = 'same', activation = 'relu')(images)
  net = Conv2D(filters =  64, kernel_size = [3,3], strides = [1,1], padding = 'same', activation = 'relu')(net)
  net = Conv2D(filters =  64, kernel_size = [3,3], strides = [1,1], padding = 'same', activation= 'relu')(net)
  net = MaxPooling2D(pool_size= (2,2))(net)
  net = Conv2D(filters = 128, kernel_size = [3,3], strides = [1,1], padding = 'same', activation = 'relu')(net)
  net = Conv2D(filters = 128, kernel_size = [3,3], strides = [1,1], padding = 'same', activation = 'relu')(net)
  net = Conv2D(filters = 128, kernel_size = [3,3], strides = [1,1], padding = 'same', activation = 'relu')(net)
  net = MaxPooling2D(pool_size= (2,2))(net)
  net = Conv2D(filters = 256, kernel_size = [3,3], strides = [1,1], padding = 'same', activation = 'relu')(net)
  net = Conv2D(filters = 256, kernel_size = [3,3], strides = [1,1], padding = 'same', activation = 'relu')(net)
  net = Conv2D(filters = 256, kernel_size = [3,3], strides = [1,1], padding = 'same', activation = 'relu')(net)
  net = Dropout(0.25)(net)
  net = AveragePooling2D(pool_size=(8,8))(net)
  net = Flatten()(net)
  net = Dense(units = 10, activation = 'softmax')(net)

  model = Model(inputs = images, outputs = net)

  return model


In [23]:
input_shape = (32,32,3)
model = MiniModel(input_shape)

In [24]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 conv2d_10 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 conv2d_11 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 16, 16, 128)       73856     
                                                           

In [25]:
#the learning schedule
def lr_schedule(epoch):
  lr = 0.01

  if epoch > 15:
    lr = lr/100

  elif epoch >10:
    lr = lr/ 10

  elif epoch > 5:
    lr = lr/ 5

  print('Learning Rate: ', lr)

  return lr

In [26]:
#pass the scheduler function into the Learning Rate Schedule

lr_scheduler = LearningRateScheduler(lr_schedule)

In [27]:
#directory to create models
save_directory = os.path.join(os.getcwd(), 'cifar10savedmodels')

#name of model files
model_name = 'cifar10model.{epoch:03d}.h5'

#create directory if it doesn't exist
if not os.path.isdir(save_directory):
  os.makedirs(save_directory)


#join the directory with the model file
modelpath = os.path.join(save_directory, model_name)

In [28]:
checkpoint = ModelCheckpoint(filepath= modelpath,
                             monitor= 'val_acc',
                             verbose= 1,
                             save_best_only= True,
                             period = 1)

In [29]:
#specify the training component
model.compile(optimizer= Adam(lr_schedule(0)), loss = 'categorical_crossentropy', metrics = ['accuracy'])

Learning Rate:  0.01


In [30]:
model.fit(train_x, train_y, batch_size = 128, epochs= 20, verbose = 1, validation_split= 0.1, callbacks= [checkpoint, lr_scheduler])

In [21]:
#Evaluate the accuracy of the test dataset
accuracy = model.evaluate(x=test_x,y=test_y,batch_size=128)
print("Accuracy: ",accuracy[1])

79/79 [==============================] - 86s 1s/step - loss: 1.1700 - accuracy: 0.5875
Accuracy:  0.5874999761581421
